In [1]:
from the_project_tracker.core.github_retriever import AbstractRetriever


class SphinxDocumentationRetriever(AbstractRetriever):
    url: str
    
    def retrieve(self, max_pr_num: int) -> list | None:
        return self.get_last_n_merged_prs_info(max_pr_num)

In [2]:
import requests
from bs4 import BeautifulSoup

url = 'https://pola-rs.github.io/polars/py-polars/html/reference/index.html'
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
html = response.content

In [3]:
soup = BeautifulSoup(html)

In [4]:
table = soup.find('li', attrs={'class': 'stripe'})

In [6]:
soup.find("div", {"id": "sidebar-primary-item"})

In [7]:
sidebar = soup.find_all("li", attrs={"class":'toctree-l1'})
sidebar[1].find("a", attrs={"class":'reference internal'}).text

'Series'

In [35]:
sidebar = soup.find("div", attrs={"class":'sidebar-primary-item'})

In [9]:
result = dict(zip([i.text for i in soup.select('h2 a')], [i.text for i in soup.find("div", attrs={"class":'sidebar-primary-item'})]))
result

{}

In [12]:
sidebar.find_all("li", attrs={"class":'toctree-l1'})[0].nextSibling

'\n'

## API methods dict

### Functionality-Method dict

In [13]:
sidebar = soup.find_all("li", attrs={"class":'toctree-l1'})
for element in sidebar:
    element.find("a", attrs={"class":'reference internal'}).text
    

In [32]:
#{i.find("a", attrs={"class":'reference internal'}).text: i.find_all("li", attrs={"class":'toctree-l2'}) for i in soup.find_all("li", attrs={"class":'toctree-l1'})}

### Method-Details dict

In [15]:
doc_dict = {}

# Select the sidebar element
sidebar = soup.find("div", attrs={"class":'sidebar-primary-item'})

# Iterate over TOC level 1
for functionality in sidebar.find_all("li", attrs={"class":'toctree-l1'}):
    functionallity_name = functionality.find("a", attrs={"class":'reference internal'}).text
    if 'has-children' in functionality.attrs['class']:
        # Initialize final dict TOC 1 level and level 2 dict blank to be filled if needed
        doc_dict[functionallity_name] = {}
        child_dict = {}
        # # Iterate over TOC level 2
        for method in functionality.find_all("li", attrs={"class":'toctree-l2'}):
            method_name = method.find("a", attrs={"class":'reference internal'}).text
            # If there is a TOC level 3, create a as a value a nested dict {method: url}
            if 'has-children' in method.attrs['class']:
                method_child_name = method.find("a", attrs={"class":'reference internal'}).text
                child_dict[method_child_name] = {}
                for child in method.find_all("li", attrs={"class":'toctree-l3'}):
                    child_name = child.find("a", attrs={"class":'reference internal'}).text
                    child_url = child.find("a", attrs={"class":'reference internal'}, href=True).attrs.get('href')
                    child_dict[method_child_name].update({child_name: child_url})
                doc_dict[functionallity_name].update({method_name: child_dict})
            # If there is not a TOC level 3, just add the url as value
            else:
                method_url = method.find("a", attrs={"class":'reference internal'}, href=True).attrs.get('href')
                doc_dict[functionallity_name].update({method_name: method_url})
    else:
        functionality_url = functionality.find("a", attrs={"class":'reference internal'}, href=True).attrs.get('href')
        doc_dict.update({functionallity_name: functionality_url})

In [16]:
def get_all_urls(dd):
    all_urls = set()
    for _, value in dd.items():
        if isinstance(value, str):
            all_urls.add(value)
        else:  # Assuming value is a nested dictionary
            all_urls.update(get_all_urls(value))
    return all_urls

In [18]:
base_url = 'https://pola-rs.github.io/polars/py-polars/html/reference/'
base_url + list(get_all_urls(doc_dict))[1]

'https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.extend_constant.html'

In [19]:
#import time 
#from tqdm.auto import tqdm
#
#content_dict = {}
#with tqdm():
#    for url in list(get_all_urls(doc_dict)):
#        method_url = base_url + url
#        response = requests.get(method_url, headers={'User-Agent': 'Mozilla/5.0'})
#        html = response.content
#        method_soup = BeautifulSoup(html)
#        content = method_soup.find("article", attrs={"class":'bd-article'}).text.strip('\n')
#        content_dict.update({url: content})
#        time.sleep(0.2)
#

In [20]:
import pickle

#with open('data/content_dict.pickle', 'wb') as handle:
#    pickle.dump(content_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('data/content_dict.pickle', 'rb') as handle:
    content_dict = pickle.load(handle)

In [21]:
print(len(content_dict))
import sys
sys.getsizeof(content_dict)

1167


36960

In [22]:
import sys
from numbers import Number
from collections import deque
from collections.abc import Set, Mapping


ZERO_DEPTH_BASES = (str, bytes, Number, range, bytearray)


def getsize(obj_0):
    """Recursively iterate to sum size of object & members."""
    _seen_ids = set()
    def inner(obj):
        obj_id = id(obj)
        if obj_id in _seen_ids:
            return 0
        _seen_ids.add(obj_id)
        size = sys.getsizeof(obj)
        if isinstance(obj, ZERO_DEPTH_BASES):
            pass # bypass remaining control flow and return
        elif isinstance(obj, (tuple, list, Set, deque)):
            size += sum(inner(i) for i in obj)
        elif isinstance(obj, Mapping) or hasattr(obj, 'items'):
            size += sum(inner(k) + inner(v) for k, v in getattr(obj, 'items')())
        # Check for custom object instances - may subclass above too
        if hasattr(obj, '__dict__'):
            size += inner(vars(obj))
        if hasattr(obj, '__slots__'): # can have __slots__ with __dict__
            size += sum(inner(getattr(obj, s)) for s in obj.__slots__ if hasattr(obj, s))
        return size
    return inner(obj_0)




In [30]:
def count_char_len(dd):
    return sum([count_char_len(v) if isinstance(v, dict) else len(v) for v in dd.values()])
def count_words(dd):
    return sum([count_words(v) if isinstance(v, dict) else len(v.split(' ')) for v in dd.values()])

count_char_len(content_dict) 
count_words(content_dict)

199974

In [31]:
list(content_dict.keys())[0].rstrip('.html').split('.')[-1]
objects = []
methods = []
for method in content_dict:
    splt = method.strip('html').split('.')
    if len(splt) >= 4:
        objects.append('.'.join(splt[1:-2]))
        methods.append(splt[-2])
    elif len(splt) == 3:
        objects.append(None)
        methods.append(splt[-2])

In [27]:
from dotenv import load_dotenv
from the_project_tracker.core.data_models import Documentation
from the_project_tracker.db.pg_conn import PGDataConnection, SettingsSSH
from datetime import datetime
load_dotenv()

db = PGDataConnection(ssh_config=SettingsSSH())

for method in content_dict:
    splt = method.rstrip('html').split('.')
    if len(splt) >= 4:
        obj = '.'.join(splt[1:-2])
        mthd= splt[-2]
    elif len(splt) == 3:
        obj = None
        mthd = splt[-2]

    doc = Documentation(
        project_name='polars',
        project_version='0.19.0',
        doc_obj=obj,
        doc_method=mthd,
        doc_content=content_dict[method],
        doc_content_interpretation=None,
        inserted_at=datetime.utcnow(),
        updated_at=datetime.utcnow(),
    )
    db.insert_documentation(doc)

Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentation inserted
Documentati